In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_json("/home/ayesha.ishaq/Desktop/AI701/CookBookCreater/recipe1M/layer1.json")
df.head()

,ingredients,url,partition,title,id,instructions
0,"[{'text': '6 ounces penne'}, {'text': '2 cups ...",http://www.epicurious.com/recipes/food/views/-...,train,Worlds Best Mac and Cheese,000018c8a5,[{'text': 'Preheat the oven to 350 F. Butter o...
1,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",http://cookeatshare.com/recipes/dilly-macaroni...,train,Dilly Macaroni Salad Recipe,000033e39b,[{'text': 'Cook macaroni according to package ...
2,"[{'text': '8 tomatoes, quartered'}, {'text': '...",http://www.foodnetwork.com/recipes/gazpacho1.html,train,Gazpacho,000035f7ed,[{'text': 'Add the tomatoes to a food processo...
3,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",http://www.food.com/recipe/crunchy-onion-potat...,test,Crunchy Onion Potato Bake,00003a70b1,[{'text': 'Preheat oven to 350 degrees Fahrenh...
4,[{'text': '1 (3 ounce) package watermelon gela...,http://www.food.com/recipe/cool-n-easy-creamy-...,train,Cool 'n Easy Creamy Watermelon Pie,00004320bb,"[{'text': 'Dissolve Jello in boiling water.'},..."


In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
import re


# Download the WordNet lemmatizer data (only need to do this once)
# nltk.download('wordnet')

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()
ignore_words = [
   'easy','recipe',
    "really",
    "breakfast",
    "quick",
    "foolproof",
    "the",
    "lunch",
    "simple",
    "comfort",
    "lovers",
    "friendly",
    "weeknight",
    "time",
    "miracle",
    "best",
    "everything",
    "heaven",
    "baba",
    "boy",
    "dirt",
    "bliss",
    "wonderful",
    "momma",
    "fresh",
    "version",
    "just",
    "thumbprint",
    "preserved",
    "lover",
    "tips",
    "city",
    "colorful",
    "your",
    "bill",
    "joy",
    "treat",
    "substitute",
    "impossible",
    "shortcut",
    "company",
    "market",
    "poor",
    "style",
    "new",
    "elegent",
    "refreshing",
    "dessert",
    "better",
    "killer",
    "game",
    "sunrise",
    "night",
    "fool",
    "fantastic",
    "uncle",
    "basic",
    "almost",
    "original",
    "another",
    "mrs",
    "fabulous",
    "speedy",
    "mom",
    "moms",
    "mother",
    "granny",
    "grandmas",
    "our",
    "secret",
    "fall",
    "crazy",
    "dirty",
    "ingredient",
    "fix",
    "copycat",
    "country",
    "birthday",
    "dad",
    "simply",
    "easiest",
    "chef",
    "lazy",
    "grandma",
    "aunt",
    "authentic",
    "mama",
    "supper",
    "classic",
    "healthy",
    "healthier",
    "heavenly",
    "magic",
    "slow",
    "ww",
    "favorite",
    "great",
    "ii",
    "iii",
    "iv",
    "yummy",
    "tasty",
    "party",
    "ultimate",
    "ever",
    "halloween",
    "diabetic",
    "very",
    "day",
    "old",
    "awesome",
    "thanksgiving",
    "amazing",
    "famous",
    "morning",
    "my",
    "fashioned",
    "holiday",
    "oldfashioned",
    "dinner",
    "minute",
    "surprise",
    "food",
    "good",
    "special",
    "key",
]

def remove_unnecessary_words_and_characters(title):
    new=[]
    for word in title.split():
        if len(word)>1 and word not in new and word not in ignore_words:
            new.append(word)

            # new.append(get_singular(word))
    return " ".join(new)

def clean_title(text):
    
    text = text.lower()
    text =  remove_unnecessary_words_and_characters(text)

    words = nltk.word_tokenize(text)  # Tokenize the sentence into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    text = " ".join(lemmatized_words)  # Join the words back into a sentence
    
    text=re.sub(r'[:\*+,-/:;<=>@\\^_`(){|}~]+',' ',text)

    text=text.replace(r'&amp;?',r' and ')
    text=text.replace(r'amp', r' and ')

    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove non-alphabetic characters
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove trailing whitespace
    text = re.sub(r'\d+', '', text)
    return text

In [4]:
def process_instructions(dfinstructions):
    instructions=''
    for text in dfinstructions:
        instructions+=text['text']
        instructions+=' '
    return clean_title(instructions)

def replace_numbers(text):
    # Replace fractions with words
    fractions = {"1/4": "quarter", "1/2": "half", "3/4": "three quarters"}
    for fraction, word in fractions.items():
        text = re.sub(r"\b{}\b".format(fraction), word, text)
    # Replace numbers with words
    number_words = {1: "one", 2: "two", 3: "three", 4: "four", 5: "five", 6: "six", 7: "seven", 8: "eight", 9: "nine", 10: "ten", 11: "eleven", 12: "twelve", 13: "thirteen", 14: "fourteen", 15: "fifteen", 16: "sixteen", 17: "seventeen", 18: "eighteen", 19: "nineteen", 20: "twenty", 30: "thirty", 40: "forty", 50: "fifty", 60: "sixty", 70: "seventy", 80: "eighty", 90: "ninety", 100: "one hundred"}
    for num, word in number_words.items():
        text = re.sub(r"\b{}\b".format(num), word, text)
    return text

def clean_ingredients(ingredient):
    ingredient_text=''
    for text in ingredient:
        ingredient_text+=text['text']
        ingredient_text+=' '
    
    text = ingredient_text.lower()
    words = nltk.word_tokenize(text)  # Tokenize the sentence into words
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # Lemmatize each word
    text = " ".join(lemmatized_words)  # Join the words back into a sentence
    text=text.replace(r'&amp;?',r' and ')
    text=text.replace(r'amp', r' and ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
df['sub_col'] = df.title.apply(lambda x:len(str(x).split()))
small_title_df = df[df["sub_col"]<=4]

small_title_df.head(20)

,ingredients,url,partition,title,id,instructions,sub_col
1,"[{'text': '1 c. elbow macaroni'}, {'text': '1 ...",http://cookeatshare.com/recipes/dilly-macaroni...,train,Dilly Macaroni Salad Recipe,000033e39b,[{'text': 'Cook macaroni according to package ...,4
2,"[{'text': '8 tomatoes, quartered'}, {'text': '...",http://www.foodnetwork.com/recipes/gazpacho1.html,train,Gazpacho,000035f7ed,[{'text': 'Add the tomatoes to a food processo...,1
3,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",http://www.food.com/recipe/crunchy-onion-potat...,test,Crunchy Onion Potato Bake,00003a70b1,[{'text': 'Preheat oven to 350 degrees Fahrenh...,4
5,"[{'text': '12 cup shredded coconut'}, {'text':...",http://www.food.com/recipe/easy-tropical-beef-...,train,Easy Tropical Beef Skillet,0000631d90,"[{'text': 'In a large skillet, toast the cocon...",4
7,"[{'text': '6 -8 cups fresh rhubarb, or'}, {'te...",http://www.food.com/recipe/strawberry-rhubarb-...,train,Strawberry Rhubarb Dump Cake,00007bfd16,[{'text': 'Put ingredients in a buttered 9 x 1...,4
8,"[{'text': '8 ounces, weight Light Fat Free Van...",http://tastykitchen.com/recipes/breakfastbrunc...,train,Yogurt Parfaits,000095fc1d,[{'text': 'Layer all ingredients in a serving ...,2
9,"[{'text': '2 cups flour'}, {'text': '1 tablesp...",http://www.food.com/recipe/zucchini-nut-bread-...,train,Zucchini Nut Bread,0000973574,"[{'text': 'Sift dry ingredients.'}, {'text': '...",3
12,"[{'text': '1 (750 ml) bottle rose wine, chille...",http://www.food.com/recipe/pink-sangria-305385,train,Pink Sangria,0000c79afb,"[{'text': 'In a pitcher, combine all the ingre...",2
13,"[{'text': '14 cup butter'}, {'text': '34 cup b...",http://www.food.com/recipe/pineapple-loaf-88070,train,Pineapple Loaf,0000ed95f8,"[{'text': 'Cream butter, sugar and egg.'}, {'t...",2
14,"[{'text': '200 grams Cake flour'}, {'text': '1...",https://cookpad.com/us/recipes/156014-brown-su...,train,Brown Sugar 'Karintou' Snacks,00010379bf,"[{'text': 'In a bowl, mix flour, baking powder...",4


In [6]:
small_title_df['clean_title'] = small_title_df.title.apply(clean_title)


/tmp/ipykernel_661852/766126274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_title_df['clean_title'] = small_title_df.title.apply(clean_title)


In [7]:
small_title_df['clean_ingredients'] = small_title_df.ingredients.apply(clean_ingredients)


KeyboardInterrupt: 

In [ ]:
small_title_df['clean_instructions'] = small_title_df.instructions.apply(process_instructions)

In [8]:
image_names = pd.read_json("/home/ayesha.ishaq/Desktop/AI701/CookBookCreater/recipe1M/layer2.json")

In [9]:
joined_df = pd.merge(small_title_df, image_names, on='id')

In [10]:
joined_df.head()

,ingredients,url,partition,title,id,instructions,sub_col,clean_title,images
0,"[{'text': '2 12 cups milk'}, {'text': '1 12 cu...",http://www.food.com/recipe/crunchy-onion-potat...,test,Crunchy Onion Potato Bake,00003a70b1,[{'text': 'Preheat oven to 350 degrees Fahrenh...,4,crunchy onion potato bake,"[{'id': '3e233001e2.jpg', 'url': 'http://img.s..."
1,"[{'text': '6 -8 cups fresh rhubarb, or'}, {'te...",http://www.food.com/recipe/strawberry-rhubarb-...,train,Strawberry Rhubarb Dump Cake,00007bfd16,[{'text': 'Put ingredients in a buttered 9 x 1...,4,strawberry rhubarb dump cake,"[{'id': '6409eab844.jpg', 'url': 'http://img.s..."
2,"[{'text': '8 ounces, weight Light Fat Free Van...",http://tastykitchen.com/recipes/breakfastbrunc...,train,Yogurt Parfaits,000095fc1d,[{'text': 'Layer all ingredients in a serving ...,2,yogurt parfait,"[{'id': 'a1374cdd98.jpg', 'url': 'http://tasty..."
3,"[{'text': '1 (750 ml) bottle rose wine, chille...",http://www.food.com/recipe/pink-sangria-305385,train,Pink Sangria,0000c79afb,"[{'text': 'In a pitcher, combine all the ingre...",2,pink sangria,"[{'id': '2f4b4c4452.jpg', 'url': 'http://img.s..."
4,"[{'text': '200 grams Cake flour'}, {'text': '1...",https://cookpad.com/us/recipes/156014-brown-su...,train,Brown Sugar 'Karintou' Snacks,00010379bf,"[{'text': 'In a bowl, mix flour, baking powder...",4,brown sugar karintou snack,"[{'id': 'f480145da5.jpg', 'url': 'https://img-..."


In [ ]:
new_train_part = joined_df[joined_df["partition"]=="train"]

In [ ]:
# Create annotation jsons for training and testing 

train_ann = []
for ind in new_train_part.index:
    for each in new_train_part['images'][ind]:
        temp_dict = {}
        temp_dict['caption'] = new_train_part['clean_title'][ind]
        img_id = each['id']
        img_path = "train/{}/{}/{}/{}/{}".format(img_id[0], img_id[1], img_id[2], img_id[3], img_id)
        temp_dict['image'] = img_path
        train_ann.append(temp_dict)

In [ ]:
FILE_PATH = './train.json'

with open(FILE_PATH, 'w') as output_file:
	json.dump(train_ann, output_file)

In [ ]:
new_test_part = joined_df[joined_df["partition"]=="test"]

In [ ]:
# Create annotation jsons for training and testing 

test_ann = []
for ind in new_test_part.index:
    for each in new_test_part['images'][ind]:
        temp_dict = {}
        temp_dict['caption'] = new_test_part['clean_title'][ind]
        img_id = each['id']
        img_path = "test/{}/{}/{}/{}/{}".format(img_id[0], img_id[1], img_id[2], img_id[3], img_id)
        temp_dict['image'] = img_path
        test_ann.append(temp_dict)

In [ ]:
FILE_PATH = './test.json'

with open(FILE_PATH, 'w') as output_file:
	json.dump(test_ann, output_file)

In [11]:
new_val_part = joined_df[joined_df["partition"]=="val"]

In [ ]:
val_ann = []
for ind in new_val_part.index:
    for each in new_val_part['images'][ind]:
        temp_dict = {}
        temp_dict['caption'] = new_val_part['clean_title'][ind]
        img_id = each['id']
        img_path = "val/{}/{}/{}/{}/{}".format(img_id[0], img_id[1], img_id[2], img_id[3], img_id)
        temp_dict['image'] = img_path
        val_ann.append(temp_dict)
        

In [ ]:
FILE_PATH = './val.json'

with open(FILE_PATH, 'w') as output_file:
	json.dump(val_ann, output_file)

Prepare annotation for evaluating recipes generated by LLMs

In [14]:
val_un = new_val_part.drop_duplicates(subset=['clean_title'])

In [18]:
val_un.head()

,ingredients,url,partition,title,id,instructions,sub_col,clean_title,images,clean_instructions,clean_ingredients
14,"[{'text': '1 12 cups whole wheat flour'}, {'te...",http://www.food.com/recipe/whole-wheat-waffles...,val,Whole Wheat Waffles,0004c091a0,"[{'text': '1.In a large bowl, mix together flo...",3,whole wheat waffle,"[{'id': 'a97924d704.jpg', 'url': 'http://img.s...",in large bowl mix together flour salt baking p...,1 12 cup whole wheat flour 3 teaspoon baking p...
18,"[{'text': '1 cup all-purpose flour'}, {'text':...",http://allrecipes.com/recipe/frosty-strawberry...,val,Frosty Strawberry Squares,000507ca6b,[{'text': 'Preheat oven to 350 degrees F (175 ...,3,frosty strawberry square,"[{'id': '8e98aee7e1.jpg', 'url': 'http://image...",preheat oven to degree c make crust combine fl...,1 cup all-purpose flour 1/2 cup chopped walnut...
24,"[{'text': '34 cup creamy peanut butter'}, {'te...",http://www.food.com/recipe/irresistible-peanut...,val,Irresistible Peanut Butter Cookies,0006354bc3,"[{'text': 'Combine peanut butter, Crisco, brow...",4,irresistible peanut butter cooky,"[{'id': '7027a5c4f9.jpg', 'url': 'http://img.s...",combine peanut butter crisco brown sugar milk ...,34 cup creamy peanut butter 12 cup crisco shor...
30,[{'text': '1 lb Ground beef (grass fed is best...,https://cookpad.com/us/recipes/367936-joes-pas...,val,Joe's Pasta E Fagiolo,00081bb5ef,"[{'text': 'In stockpot, add the oil & heat on ...",4,joe s pasta fagiolo,"[{'id': '1fd9fdddd4.jpg', 'url': 'https://img-...",in stockpot add oil heat on medium onion garli...,1 lb ground beef ( grass fed is best or all na...
32,"[{'text': '1 lb chicken wing pieces'}, {'text'...",https://cookpad.com/us/recipes/337237-vietname...,val,Vietnamese chicken wings,000875a149,"[{'text': 'Heat canola oil, lightly flour chic...",3,vietnamese chicken wing,"[{'id': '0c901c0aee.jpg', 'url': 'https://img-...",heat canola oil lightly flour chicken and fry ...,1 lb chicken wing piece 1 1/2 cup cilantro 1 c...


In [20]:
val_ann = []
for ind in val_un.index:
    temp_dict = {}
    temp_dict['recipe'] = "ingredients: "+ val_un['clean_ingredients'][ind] + " instructions: "+ val_un['clean_instructions'][ind]
    temp_dict['title'] = val_un['clean_title'][ind]
    val_ann.append(temp_dict)

In [ ]:
FILE_PATH = './val_recipe.json'

with open(FILE_PATH, 'w') as output_file:
	json.dump(val_ann, output_file)